In [13]:
import lmdb
import os
import copy
import torch
import random
import skimage
import numpy as np
import torch.nn as nn
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

torch.cuda.set_device(2)
print("complete")

complete


In [14]:
local_train = '/home/lvlab/Pytorch/object_bird_train.npy'
local_test = '/home/lvlab/Pytorch/object_bird_test.npy'
train_vectors = np.load("part_bird_train.npy")
test_vectors = np.load("part_bird_test.npy")
print("complete")

complete


In [15]:
class Bird_Dataset_Process(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, train_vectors, num, local_file, transform=None, transform2=None, 
                 transform3=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.root_dir = root_dir
        self.transform = transform
        self.train_vectors = train_vectors
        self.part_num = num
        self.local_points = np.load(local_file)
        self.transform2 = transform2
        self.transform3 = transform3

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
        
        sample = {}
        sample['image'] = copy.deepcopy(image)
        if self.transform:
            sample['image'] = self.transform2(sample['image'])
        
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448))) 
        label = int(img_dir_label[1])
        
        m1, m2, n1, n2, x1, x2, y1, y2, max_x, max_y = self.local_points[idx, :]


        x1, x2, y1, y2 = np.min(obj_vec[:, 0]), np.max(obj_vec[:, 0]), \
                        np.min(obj_vec[:, 1]), np.max(obj_vec[:, 1])
        edge = 16
        
        
        l = np.maximum(x2 - x1, y2 - y1) * 8 + edge
        x1, x2, y1, y2 = np.maximum(0, core_x - l), np.minimum(448, core_x + l), \
                    np.maximum(0, core_y - l), np.minimum(448, core_y + l)
        
        obj = image[m1:m2, n1:n2]
        sample['object'] = copy.deepcopy(obj)
        if self.transform3:
            sample['object'] = self.transform3(sample['object'])
        
        vectors = self.train_vectors[idx]
        new_vectors = []
        for i in range(self.part_num):
            vectors_part = vectors[i]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)
            num_p = vectors_part.shape[0]
#             print(num_p)
            cls_p = 0
            lists = []
            lab_set = {i for i in range(num_p)}
            while(cls_p < num_p):
                cls_l = []
                start = lab_set.pop()
                cls_l.append(start)
                cls_p += 1
                cls_new_p = True
                while(cls_new_p):
                    rm_l = []
                    cls_new_p = False
                    for k in lab_set:
                        new_p = False
                        for t in range(len(cls_l)):
#                             print(pow(vectors_part[k, 0:2] - vectors_part[cls_l[t], 0:2], 2))
                            dis = np.sum(pow(vectors_part[k, 0:2] - vectors_part[cls_l[t], 0:2], 2)) 
                            if dis <= 2:
                                new_p = True
                                break
                        if new_p:
                            cls_new_p = True
                            rm_l.append(k)
                            cls_l.append(k)
                            cls_p += 1

                    for kk in range(len(rm_l)):
                        lab_set.remove(rm_l[kk])
                lists.append(cls_l)
#             print(len(lists))
            l_n = [len(lists[i]) for i in range(len(lists))]
#             print(l_n)
            new_vector_index = lists[int(l_n.index(max(l_n)))]
            new_vector = np.zeros((max(l_n), 4))
            for i in range(max(l_n)):
                new_vector[i] = vectors_part[new_vector_index[i]]
            new_vectors.append(new_vector)
        new_vectors = np.array(new_vectors)
        
        weights = np.zeros(self.part_num)
        points = np.zeros((self.part_num, 2))
        for i in range(self.part_num):
            vectors_part = new_vectors[i]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)
            weights[i] = np.sum(vectors_part[:, 2])/vectors_part.shape[0]
            points[i, 0] = np.sum(vectors_part[:, 0])/vectors_part.shape[0]
            points[i, 1] = np.sum(vectors_part[:, 1])/vectors_part.shape[0]
            
        index = np.argsort(weights)

        for i in range(self.part_num):
            vectors_part = new_vectors[index[i]]
            if isinstance(vectors_part, list):
                vectors_part = np.array(vectors_part)

            x, y = np.round(points[index[i], 0] * 16 + 8), np.round(points[index[i], 1] * 16 + 8)
            
            l_d = np.maximum((np.max(vectors_part[:, 0]) - np.min(vectors_part[:, 0])), \
                            (np.max(vectors_part[:, 1]) - np.min(vectors_part[:, 1])))

            if l_d <= 4:
                l = 48
            elif 4 < l_d <= 6:
                l = 64
            elif 6 < l_d <= 8:
                l = 80
            elif 8 < l_d <= 10:
                l = 96
            elif 10 < l_d <= 12:
                l = 112
            elif 12 < l_d <= 14:
                l = 128
            else:
                l = 144

            m1, m2, n1, n2 = int(np.maximum(0, x - l)), int(np.minimum(448, x + l)), \
                            int(np.maximum(0, y - l)), int(np.minimum(448, y + l))

            sample['image%d' % (self.part_num - i - 1)] = image[m1:m2, n1:n2]            
        
        sample['label'] = label

        if self.transform:
            for i in range(self.part_num):
                sample['image%d' % i] = self.transform(sample['image%d' % i])

        return sample

In [16]:
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

data_transform3 = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(224),
                transforms.RandomCrop(224),
#                 transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
            ])

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([448, 448]),
        transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize([0.486,0.499,0.432], [0.229,0.225,0.263])
    ]),
}

root_dir = '/home/lvlab/Pytorch/images'
train_file = '/home/lvlab/Pytorch/train.txt'
test_file = '/home/lvlab/Pytorch/test.txt'

img_data = {'train': Bird_Dataset_Process(train_file, root_dir, train_vectors, 4, local_train, 
                                          data_transform, data_transforms['train'], data_transform3),
                  'test': Bird_Dataset_Process(test_file, root_dir, test_vectors, 4, local_test, 
                                            data_transform, data_transforms['test'], data_transform3)}
loader_img = {x: torch.utils.data.DataLoader(img_data[x], batch_size=2,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(img_data[x]) for x in ['train', 'test']}
print(img_data['train'][1]['image'].shape)

torch.Size([3, 448, 448])


/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/lvlab/.tensorflow_pytorch/lib/python3.5/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [17]:
def test_model(model_part, model_obj, model_img, criterion):
    start_time = time.time() 
    aa, bb, cc = 1, 1, 1
    running_loss = 0.0
    running_corrects = 0
    print("********************************************************")
    print("aa = ", aa, " bb = ", bb, " cc = ", cc)
    # Iterate over data.
    for i_batch, sample_batched in enumerate(loader_img['test']):
        img0 = sample_batched['image0']
        img1 = sample_batched['image1']
        img2 = sample_batched['image2']
        img3 = sample_batched['image3']
        obj = sample_batched['object']
        original = sample_batched['image']
        labels = sample_batched['label']

        obj = obj.to(device)
        original = original.to(device)
        img0 = img0.to(device)
        img1 = img1.to(device)
        img2 = img2.to(device)
        img3 = img3.to(device)
        labels = labels.to(device)

        output = cc * (model_part(img0) + model_part(img1) + model_part(img2) + model_part(img3)) +  \
                aa * model_img(original) + bb * model_obj(obj)
        _, preds = torch.max(output, 1)
        loss = criterion(output, labels)


        # statistics
        running_loss += loss.item() * img0.size(0)
        running_corrects += torch.sum(preds == labels.data)

    phase = 'test'
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))

    cost_time = (time.time() - start_time)/60.0
    print('{} 1 epoch time: {:.2f}min'.format(
        phase, cost_time))

    return 0

In [18]:
import time
import copy
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:0")

model_part = torch.load('/home/lvlab/Pytorch/part_bird')
model_obj = torch.load('/home/lvlab/Pytorch/object_bird')
model_img = torch.load('/home/lvlab/Pytorch/image_bird')

model_part = model_part.to(device)
model_obj = model_obj.to(device)
model_img = model_img.to(device)

criterion = nn.CrossEntropyLoss()

test_model(model_part, model_obj, model_img, criterion)

********************************************************
aa =  1  bb =  1  cc =  1
test Loss: 2.3798 Acc: 0.8441
test 1 epoch time: 3.38min


0